# **1. Mount to Google Drive**

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

# **2. Imports**

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# ***Imports for Performance Metrics/Visualization***

In [8]:
# Imports for Performance Metrics/Visualization
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    f1_score, precision_recall_curve, PrecisionRecallDisplay,
    accuracy_score, roc_curve, RocCurveDisplay
)
from sklearn.preprocessing import label_binarize

# **3. Setup Directories to Training, Validation, and Testing**

In [11]:
base_dir = 'G:/.shortcut-targets-by-id/1PfaEKQoN9ziSq5ctd1Kwcm8wcEnWPruy/CAPSTONE 2024 2025/COE70B - Sem 2/Phase 1/Prayash/Emotions'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

print("Training Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Testing Directory:", test_dir)

# Directory to save augmented images
augmented_train_dir = os.path.join(base_dir, 'augmented_train')

# Create the augmented directory if it doesn't exist
if not os.path.exists(augmented_train_dir):
    os.makedirs(augmented_train_dir)

Training Directory: G:/.shortcut-targets-by-id/1PfaEKQoN9ziSq5ctd1Kwcm8wcEnWPruy/CAPSTONE 2024 2025/COE70B - Sem 2/Phase 1/Prayash/Emotions\train
Validation Directory: G:/.shortcut-targets-by-id/1PfaEKQoN9ziSq5ctd1Kwcm8wcEnWPruy/CAPSTONE 2024 2025/COE70B - Sem 2/Phase 1/Prayash/Emotions\validation
Testing Directory: G:/.shortcut-targets-by-id/1PfaEKQoN9ziSq5ctd1Kwcm8wcEnWPruy/CAPSTONE 2024 2025/COE70B - Sem 2/Phase 1/Prayash/Emotions\test


# **4. Data Augmentation and Saving Augmented Images**

In [13]:
# Define the augmentation pipeline
augmentation_pipeline = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20%
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20%
    shear_range=0.2,  # Apply shearing transformations
    zoom_range=0.2,  # Randomly zoom in/out by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    brightness_range=[0.8, 1.2],  # Adjust brightness
    fill_mode='nearest'  # Fill in missing pixels after transformations
)

# Number of augmented copies to create per image
num_augmented_copies = 5

# Check if the augmented_train directory already exists and contains images
if os.path.exists(augmented_train_dir) and any(os.listdir(augmented_train_dir)):
    print("Augmented images already exist. Skipping augmentation.")
else:
    # Loop through each class directory
    for class_name in os.listdir(train_dir):
        class_dir = os.path.join(train_dir, class_name)
        augmented_class_dir = os.path.join(augmented_train_dir, class_name)

        # Create the augmented class directory if it doesn't exist
        if not os.path.exists(augmented_class_dir):
            os.makedirs(augmented_class_dir)

        # Loop through each image in the class directory
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)

            # Skip non-image files (e.g., hidden files like .DS_Store)
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                print(f"Skipping non-image file: {image_path}")
                continue

            try:
                # Load the image
                image = load_img(image_path)
                image_array = img_to_array(image)  # Convert to numpy array
                image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

                # Generate augmented images
                augmented_images = augmentation_pipeline.flow(
                    image_array,
                    batch_size=1,
                    save_to_dir=augmented_class_dir,  # Save to the emotion-specific folder
                    save_prefix=f'aug_{class_name}',
                    save_format='jpg'
                )

                # Save the augmented images
                for i in range(num_augmented_copies):
                    next(augmented_images)

            except UnidentifiedImageError:
                print(f"Cannot identify image file: {image_path}. Skipping.")
                continue
            except Exception as e:
                print(f"Error processing image {image_path}: {e}. Skipping.")
                continue

Augmented images already exist. Skipping augmentation.


# **5. Create Data Generators for Augmented Dataset**

In [14]:
# Data Augmentation for Training Data (already applied, so no need for additional augmentation)
train_datagen = ImageDataGenerator(rescale=1./255)

# Validation and Test Data Generators (No Augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create Generators
train_generator = train_datagen.flow_from_directory(
    augmented_train_dir,  # Use the augmented dataset
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 2616 images belonging to 8 classes.
Found 42 images belonging to 8 classes.
Found 0 images belonging to 8 classes.


# **6. Model Training**

In [ ]:
def create_transfer_learning_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)  # Increase neurons
    x = BatchNormalization()(x)  # Add Batch Normalization
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)  # Additional dense layer
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the Model
transfer_learning_model = create_transfer_learning_model()

# Train the Model (without validation set)
transfer_learning_history = transfer_learning_model.fit(
    train_generator,
    epochs=100  # Train for a fixed number of epochs
)

Epoch 1/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 52s 544ms/step - accuracy: 0.1470 - loss: 3.3522
Epoch 2/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 49s 601ms/step - accuracy: 0.1495 - loss: 3.1661
Epoch 3/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 48s 582ms/step - accuracy: 0.1943 - loss: 2.8494
Epoch 4/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 46s 556ms/step - accuracy: 0.1960 - loss: 2.8136
Epoch 5/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 45s 552ms/step - accuracy: 0.2027 - loss: 2.7365
Epoch 6/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 44s 538ms/step - accuracy: 0.2281 - loss: 2.6095
Epoch 7/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 45s 548ms/step - accuracy: 0.2363 - loss: 2.5974
Epoch 8/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 50s 611ms/step - accuracy: 0.2477 - loss: 2.4577
Epoch 9/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 48s 586ms/step - accuracy: 0.2588 - loss: 2.4153
Epoch 10/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 49s 602ms/step - accuracy: 0.2691 - loss: 2.3317
Epoch 11/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 51s 623ms/step - accuracy: 0.2855 - loss: 2.3411
Epoch 12/100
82/82 ━━━━━━━━━━━

# **7. Performance Metrics and Visualizations**

In [ ]:
# Part 1: Accuracy, Loss
train_accuracy = transfer_learning_history.history['accuracy']
#val_accuracy = transfer_learning_history.history['val_accuracy']

print(f"Training Accuracy: {train_accuracy[-1] * 100:.2f}%")
#print(f"Validation Accuracy: {val_accuracy[-1] * 100:.2f}%")

# Evaluate transfer learning model on the validation set
transfer_learning_scores = transfer_learning_model.evaluate(val_generator)
print(f"Test Loss: {transfer_learning_scores[0]:.4f}, Test Accuracy: {transfer_learning_scores[1] * 100:.2f}%")

# Part 2: Learning and Validation Curves
# Accuracy vs. Epochs
plt.figure()
plt.plot(transfer_learning_history.history['accuracy'], label='Training Accuracy', color='blue')
#plt.plot(transfer_learning_history.history['val_accuracy'], label='Validation Accuracy', color='green')
plt.title("Accuracy vs. Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc="best")
plt.show()

# Loss vs. Epochs
plt.plot(transfer_learning_history.history['loss'], label='Training Loss')
#plt.plot(transfer_learning_history.history['val_loss'], label='Validation Loss')
plt.title("Loss vs. Epochs")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.show()

# Part 3: Confusion Matrix
# Get model predictions
y_prob = transfer_learning_model.predict(val_generator)  # Probabilities for each class
y_pred = np.argmax(y_prob, axis=1)  # Predicted class indices
y_true = val_generator.labels  # True labels

emotion_indices = val_generator.class_indices  # Maps class names to indices
emotion_names = list(emotion_indices.keys())  # List of class names

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=emotion_names)
plt.figure(figsize=(10, 6))
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

# Part 4: Precision-Recall Curve
y_true_binarized = label_binarize(y_true, classes=range(len(emotion_names)))
for i, emotion in enumerate(emotion_names):
    precision, recall, _ = precision_recall_curve(y_true_binarized[:, i], y_prob[:, i])
    plt.plot(recall, precision, marker='.', label=emotion)
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.show()

# Part 5: ROC Curve
for i, emotion in enumerate(emotion_names):
    fpr, tpr, _ = roc_curve(y_true_binarized[:, i], y_prob[:, i])
    plt.plot(fpr, tpr, marker='.', label=emotion)
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

# Part 6: F1 Score (weighted), Accuracy
# F1 Score (weighted)
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score (weighted): {f1:.2f}")

# Accuracy Metric
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [17]:
model.save("emotion_model.h5")  # Saves the model in HDF5 format


NameError: name 'model' is not defined